Create a ad hoc network having 10 nodes and a total number of 32 connections.
At each node, generate the data according to the following model:

$$ y_k(n) = \boldsymbol{\theta}_0^T \mathbf{x}_k(n) + \eta_k(n), 1 \leq k \leq 10, $$

where
* $ \boldsymbol{\theta}_0 \in \mathbb{R}^{60} $ is a constant vector, generated via $ N(0,1) $.
* all input vectors $ \mathbf{x}_k(n) \in \mathbb{R}^{60} $ are i.i.d generated via $ N(0,1) $.
* noise samples $ \eta_k(n) \in \mathbb{R}^1 $ are independently generated from zero mean Gaussians with variances corresponding to different signal-to-noise level, varying from 20-25 dBs in each node.

For the unknown vector estimation employ the following algorithms:
* combine-then-adapt diffusion APSM. Parameters: $ \mu_n = 0.5 \times M_n $, $ \epsilon_k = \sqrt 2 \sigma_k $, $ q = 20 $.
* adapt-then-combine LMS. Parameter: step size = 0.03.
* combine-then-adapt LMS. Parameter: step size = 0.03.
* noncooperative LMS. Parameters: choose $ a_{mk} $ according to Metropolis rule:

$$ a_{mk} = \left\{
\begin{aligned}
&\frac{1}{max(n_k,n_m)}, &k \neq m,\ k,\ m\ are\ neighbors, \\
&0, &m \neq k, \\
&1-\sum_{i \neq k} a_{ik}, &m = k.
\end{aligned}
\right.
$$

Run 100 independent experiments and plot the average MSD per iteration in dBs:

$$ MSD(n) = 10 \log_{10} \left( \frac{1}{K} \sum_{k=1}^{K} \Vert \boldsymbol{\theta}_{k}(n) - \boldsymbol{\theta}_0 \Vert^2 \right). $$

* class networkController: generate $ \boldsymbol{\theta}_0 $ and send it to nodes, signal nodes to begin training, record training history, compute MSD.
* class node: generate input vector and noise, run algorithms, communicate with adjacent nodes.

In [1]:
# Import packages
import numpy as np
import scipy as sp
import matplotlib as plt

class networkController: generate $ \boldsymbol{\theta}_0 \in \mathbb{R}^{60} $ via $ N(0,1) $, compute true value according to sample, signal nodes to begin training, record training history, compute MSD.

In [18]:
# networkController
class networkController:
    
    def __init__(self):
        self.true_theta = np.zeros(60) # initialise theta0 in R^60
        self.energy = 0 # initialise energy level of signal
        self.nodes = [networkNode(i,self) for i in range(10)] # initialise nodelist
        self.count_nodes = len(self.nodes)
        self.history = np.array(0)
    
    # generate theta0 via standard normal distribution
    def genTheta(self):
        self.true_theta = np.random.randn(60)
        self.energy = np.linalg.norm(self.true_theta)**2
        return
    
    # train models for iters iterations using selected algorithm
    def startTrain(self, iters: int, algorithm: str):
        for node in self.nodes:
            node.adjacent_nodes = list(node.adjacent_nodes)
            node.updateArgs()
            node.noise = np.sqrt(self.energy*np.power(10,-node.snr/10))
        # combine-then-adapt ASPM
        if algorithm == "caASPM":
            return
        # adapt-then-combine LMS
        elif algorithm == "acLMS":
            self.acLMS(iters)
        # combine-then-adapt LMS
        elif algorithm == "caLMS":
            return
        # noncooperative LMS
        elif algorithm == "ncLMS":
            return
        for node in self.nodes:
            node.adjacent_nodes = set(node.adjacent_nodes)
        return
    
    # Algorithms
    # combine-then-adapt ASPM
    def caASPM(self, iters: int):
        return
        
    # adapt-then-combine LMS
    def acLMS(self, iters: int):
        for _ in range(iters):
            # sample
            for node in self.nodes:
                node.sample()
                node.y = np.dot(self.true_theta,node.x)+np.random.normal(0,node.noise)
            
            # compute error
            for node in self.nodes:
                for i in range(node.count_nodes):
                    node.error[i] = node.adjacent_nodes[i].y-np.dot(node.theta,node.adjacent_nodes[i].x)
            
            # compute psi
            for node in self.nodes:
                node.psi = np.zeros(60)
                for i in range(node.count_nodes):
                    node.psi += node.adjacent_vector[i]*node.error[i]*node.adjacent_nodes[i].x
                node.psi *= node.mu
                node.psi += node.theta
            
            # compute new theta
            for node in self.nodes:
                node.theta = np.zeros(60)
                for i in range(node.count_nodes):
                    node.theta += node.adjacent_vector[i]*node.adjacent_nodes[i].psi
            
            # compute msd
            self.computeMSD()
            
        return
        
    # combine-then-adapt LMS
    def caLMS(self, iters: int):
        return
        
    # noncooperative LMS
    def ncLMS(self, iters: int):
        return
    
    # compute msd in dBs
    def computeMSD(self):
        msd = 0
        for node in self.nodes:
            msd += np.linalg.norm(self.true_theta-node.theta)**2
        msd = 10*np.log(msd/self.count_nodes)/np.log(10)
        self.history = np.append(self.history,msd)
    
    def networkInf(self):
        count_connections = (sum(node.count_nodes for node in self.nodes)-len(self.nodes))//2
        print("This network has %d nodes with %d connections."%(len(self.nodes),count_connections))
        print("The theta vector of this network is:\n",self.true_theta)
        return

class networkNode: add and delete adjacent nodes, generate sample, train the regressor $ \theta $, compute energy of the noise.

In [19]:
# networkNode
class networkNode:
    
    def __init__(self, num: int, controller: networkController):
        self.controller = controller
        self.num = num
        
        # main arguments
        self.theta = np.zeros(60) # initialise theta in R^60
        self.mu = 0.03 # initialise mu_k
        self.x = np.random.randn(60) # initialise sample vector
        self.y = 0
        self.error = np.zeros(0) # initialise error vector
        self.psi = np.zeros(60) # initialise psi vector
        self.snr = np.random.uniform(20,25) # initialise signal-to-noise ratio
        self.noise = 0 # initialise noise
        
        # adjacent nodes
        self.adjacent_nodes = set([self]) # initialise adjacent nodelist
        self.count_nodes = 1 # count adjacent nodes
        self.adjacent_vector = np.zeros(0) # initialise matrix C and A = C
    
    # connect self and nodes in node_list
    def addAdjacentNode(self, node_list: list):
        node_list = set(node_list)
        for node in node_list:
            self.adjacent_nodes.add(node)
            node.adjacent_nodes.add(self)
            node.count_nodes = len(node.adjacent_nodes)
        self.count_nodes = len(self.adjacent_nodes)
        return
    
    # disconnect self and nodes in node_list
    def removeAdjacentNode(self, node_list: list):
        node_list = set(node_list)
        for node in node_list:
            self.adjacent_nodes.discard(node)
            node.adjacent_nodes.discard(self)
            node.count_nodes = len(node.adjacent_nodes)
        self.adjacent_nodes.add(self)
        self.count_nodes = len(self.adjacent_nodes)
        return
    
    def sample(self):
        self.x = np.random.normal(0,1,60)
        return
    
    def updateArgs(self):
        # initialise a count_nodes-dimensionial error vector
        self.error = np.zeros(self.count_nodes)
        
        # select C and A = C
        self.adjacent_vector = np.zeros(self.count_nodes)
        for i in range(self.count_nodes):
            if self.adjacent_nodes[i].num != self.num:
                self.adjacent_vector[i] = 1/np.max([self.count_nodes,self.adjacent_nodes[i].count_nodes])
            else:
                mark = i
        self.adjacent_vector[mark] = 1-np.sum(self.adjacent_vector)
                
        return

In [20]:
controller = networkController()

In [21]:
controller.nodes[0].addAdjacentNode([controller.nodes[1],controller.nodes[4],controller.nodes[6]])
controller.nodes[1].addAdjacentNode([controller.nodes[2],controller.nodes[5],controller.nodes[8]])
controller.nodes[2].addAdjacentNode([controller.nodes[3],controller.nodes[7],controller.nodes[9]])
controller.nodes[3].addAdjacentNode([controller.nodes[6],controller.nodes[7],controller.nodes[9]])

In [22]:
controller.genTheta()
controller.networkInf()
controller.energy

This network has 10 nodes with 12 connections.
The theta vector of this network is:
 [ 0.07517529 -0.97802157 -1.774246    0.01667735  1.45531114  0.8179984
 -0.04713553  0.84526242  0.28626352 -0.07761969 -0.40616353 -0.38257476
  1.4336187  -0.57744759 -0.14755126  0.15954186 -0.055384    0.92168463
  0.44535643 -1.25748153  1.2029928  -2.35139169  0.65383698 -1.63912509
 -0.56711117  0.21682048 -0.5686951  -2.07905733 -0.29771309  2.15110213
 -0.14726741  0.95137476 -0.8561074   0.38219005  0.50723223  0.68004059
  3.8204562   0.45945587  0.4886215   0.68264725 -1.18682871 -0.11668008
  2.11895167 -0.71998745  0.71483422  1.01598144 -0.84327717  2.31285706
 -0.25532281  0.80920348  1.04762873  0.42616138  0.03808871  1.59396531
  0.38765577 -0.15638793  1.66085229  0.48064368 -0.52815112 -1.17238548]


74.33276647964928

In [36]:
controller.startTrain(100,"acLMS")

In [37]:
controller.history[-1]

-11.558719063308896